In [4]:
#from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

C:\Users\jmo4cob\AppData\Local\conda\conda\envs\tf35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:
import sys
import tensorflow as tf
sys.version

'3.5.5 |Anaconda, Inc.| (default, Mar 12 2018, 17:44:09) [MSC v.1900 64 bit (AMD64)]'

In [6]:
batch_size = 128
num_classes = 10
epochs = 12

In [7]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
# Download Link - https://s3.amazonaws.com/img-datasets/mnist.npz
(x_train, y_train), (x_test, y_test) = mnist.load_data() 

In [8]:
K.image_data_format() #changes with backend - Tensorflow or Theano 

'channels_last'

In [9]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [10]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [11]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [12]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [13]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

In [19]:
print(keras.__version__)
print(tf.__version__)
#tf 3.5 and keras 2.0.8 
#latest keras version will throw an dead kernel error 
# latest tf (1.8) dont have a compatible cUDNN working with the current GPU!!..

2.0.8
1.4.0


In [20]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [21]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 22s - loss: 0.3437 - acc: 0.8955 - val_loss: 0.0822 - val_acc: 0.9758
Epoch 2/12
60000/60000 [==============================] - 19s - loss: 0.1184 - acc: 0.9653 - val_loss: 0.0549 - val_acc: 0.9822
Epoch 3/12
60000/60000 [==============================] - 19s - loss: 0.0876 - acc: 0.9741 - val_loss: 0.0434 - val_acc: 0.9859
Epoch 4/12
60000/60000 [==============================] - 18s - loss: 0.0730 - acc: 0.9787 - val_loss: 0.0403 - val_acc: 0.9874
Epoch 5/12
60000/60000 [==============================] - 19s - loss: 0.0620 - acc: 0.9815 - val_loss: 0.0365 - val_acc: 0.9878
Epoch 6/12
60000/60000 [==============================] - 18s - loss: 0.0582 - acc: 0.9821 - val_loss: 0.0335 - val_acc: 0.9893
Epoch 7/12
60000/60000 [==============================] - 19s - loss: 0.0508 - acc: 0.9843 - val_loss: 0.0312 - val_acc: 0.9896
Epoch 8/12
60000/60000 [==============================

In [22]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.027269022759463406
Test accuracy: 0.9909


In [23]:
model.save('.\model\myModel_MNIST.h5')